# Robocup Soccer Data Set - Object Tracking
**Author:** Christian Byron  **Date:** 25-May-21

This notebook provides an example of tracking robots in a video using the mean shift algorithm. The code has been modified from a tutorial by Sergio Canu

### Step 1 - Open the Robocup Soccer Video from the saved location

In [2]:
from ipywidgets import Video, Image

video_filename = 'C:\\Users\\s441606\\Documents\\Robocup_video1.mp4'
video = Video.from_file(video_filename)
video

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00#\xeemoov\x00\x00\x00lmvhd\x00\x00\…

### Step 2 - Get the location of the robot to be tracked from the first frame & establish histogram for this ROI

In [3]:
import cv2

video_cv2 = cv2.VideoCapture(video_filename)

_, first_frame = video_cv2.read()

video_cv2.release()

x = 250
y = 135
width = 70
height = 135

roi = first_frame[y: y + height, x: x + width]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])
roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

cv2.rectangle(first_frame,(x,y), (x + width, y + height), (0,255,0),2)

image_filename = 'C:\\Users\\s441606\\Documents\\Robocup_video1_first_frame.png'

cv2.imwrite(image_filename, first_frame)

image_widget = Image()

file = open(image_filename,"rb")
image_widget.value = file.read()



image_widget

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01h\x08\x02\x00\x00\x00HP\xdd\x06\x…

### Step 3 - Run through the video and track the robot using meanshift

In [3]:
term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

video_cv2 = cv2.VideoCapture('C:\\Users\\s441606\\Documents\\Robocup_video1.mp4')
frames = []

while True:
    try:
        _, frame = video_cv2.read()
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        _, track_window = cv2.meanShift(mask, (x, y, width, height), term_criteria)
        x, y, w, h = track_window
    
        cv2.rectangle(frame,(x,y), (x + w, y + h), (0,255,0),2)
    
        frames.append(frame)
    except Exception:
        break

width = int(video_cv2.get(3))
height = int(video_cv2.get(4))

filename = 'C:\\Users\\s441606\\Documents\\output.mp4'

fourcc = cv2.VideoWriter_fourcc(*'avc1')
writer = cv2.VideoWriter(filename, fourcc, 25, (width, height))

for frame in frames:
    writer.write(frame)

video_cv2.release()
writer.release()

video_output = Video.from_file(filename)
video_output

Video(value=b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00mp41isom\x00\x00\x00(uuid\\\xa7\x08\xfb2\x8eB\x05\xa8ae\…

### Step 4 - Run through the video and track the robot using Lucas-Kanade method

In [4]:
import cv2
import numpy as np

# Lucas kanade params
lk_params = dict(winSize = (15, 15),
maxLevel = 4,
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

video_cv2 = cv2.VideoCapture('C:\\Users\\s441606\\Documents\\Robocup_video1.mp4')
frames = []
width = 70
height = 135

old_points = np.array([[285, 180]], dtype=np.float32)
_, frame = video_cv2.read()
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
old_gray = gray_frame.copy()


while True:
    try:
        _, frame = video_cv2.read()
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, old_points, None, **lk_params)
        old_gray = gray_frame.copy()
        old_points = new_points

        x, y = new_points.ravel()
        
        cv2.circle(frame,(int(x),int(y)), 20, (0,255,0),2)
    
        frames.append(frame)
        
    except Exception:
        break



width = int(video_cv2.get(3))
height = int(video_cv2.get(4))

filename = 'C:\\Users\\s441606\\Documents\\output_kl.mp4'

fourcc = cv2.VideoWriter_fourcc(*'avc1')
writer = cv2.VideoWriter(filename, fourcc, 25, (width, height))

for frame in frames:
    writer.write(frame)

video_cv2.release()
writer.release()

video_output_2 = Video.from_file(filename)
video_output_2

Video(value=b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00mp41isom\x00\x00\x00(uuid\\\xa7\x08\xfb2\x8eB\x05\xa8ae\…

#### Lessons Learnt
- How to write an image to file using CV2 (i.e. ``cv2.imwrite(filepath, img)`` ) 


#### References
- https://pysource.com/2018/04/05/object-tracking-with-mean-shift-opencv-3-4-with-python-3-tutorial-29/